<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
# это у нас без 8 этапа, то есть без распаллалеливания
import torch
import torch.nn as nn
import torch.nn.functional as F

class CausalSelfAttention(nn.Module):
    # === Добавлено на этапе 3: собственная реализация каузального внимания ===
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5
        self.qkv_proj = nn.Linear(embed_dim, 3 * embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.qkv_proj(x)
        q, k, v = qkv.chunk(3, dim=-1)
        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        attn_weights = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        attn_weights = attn_weights.masked_fill(mask == 0, float('-inf'))
        attn_probs = F.softmax(attn_weights, dim=-1)
        attn_output = torch.matmul(attn_probs, v)
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, C)
        return self.out_proj(attn_output)
    # === Конец добавленного на этапе 3 кода ===

# === Добавлено на этапе 4: MLP-блок после attention ===
class FeedForward(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, 4 * embed_dim)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(4 * embed_dim, embed_dim)

    def forward(self, x):
        return self.fc2(self.act(self.fc1(x)))
# === Конец добавленного на этапе 4 кода ===


class MiniGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, max_seq_len=128, num_layers=4):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)

        # === Добавлено на этапе 2: позиционные эмбеддинги ===
        self.position_embedding = nn.Embedding(max_seq_len, embed_dim)
        self.max_seq_len = max_seq_len
        # === Конец добавленного на этапе 2 кода ===

        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads) for _ in range(num_layers)
        ])

        # === Добавлено на этапе 7: финальный LayerNorm ===
        self.final_ln = nn.LayerNorm(embed_dim)
        # === Конец добавленного на этапе 7 кода ===

        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)

        # === Добавлено на этапе 7: Weight tying ===
        self.lm_head.weight = self.token_embedding.weight
        # === Конец добавленного на этапе 7 кода ===

    def forward(self, x, targets=None):
        B, T = x.size()
        tok_emb = self.token_embedding(x)
        pos = torch.arange(0, T, device=x.device).unsqueeze(0)
        pos_emb = self.position_embedding(pos)
        x = tok_emb + pos_emb

        for block in self.blocks:
            x = block(x)

        # === Добавлено на этапе 7: финальный LayerNorm перед головой ===
        x = self.final_ln(x)
        # === Конец добавленного на этапе 7 кода ===

        logits = self.lm_head(x)

        # === Добавлено на этапе 7: расчет loss при наличии targets ===
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        return logits, loss
        # === Конец добавленного на этапе 7 кода ===

    # === Добавлено на этапе 7: функция генерации текста ===
    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.max_seq_len:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx
    # === Конец функции генерации ===

# Пример использования
model = MiniGPT(vocab_size=1000, embed_dim=64, num_heads=4, num_layers=4)
tokens = torch.randint(0, 1000, (2, 5))
targets = torch.randint(0, 1000, (2, 5))
logits, loss = model(tokens, targets)
print("Logits shape:", logits.shape)
print("Loss:", loss.item())

# Пример генерации
generated = model.generate(tokens[:, :2], max_new_tokens=5)
print("Generated tokens:", generated)


Logits shape: torch.Size([2, 5, 1000])
Loss: 44.29896926879883
Generated tokens: tensor([[477, 265, 265, 265, 265, 265, 265],
        [909, 930, 930, 930, 930, 930, 930]])


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CausalSelfAttention(nn.Module):
    # === Добавлено на этапе 3: собственная реализация каузального внимания ===
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5
        self.qkv_proj = nn.Linear(embed_dim, 3 * embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.qkv_proj(x)
        q, k, v = qkv.chunk(3, dim=-1)
        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        attn_weights = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        attn_weights = attn_weights.masked_fill(mask == 0, float('-inf'))
        attn_probs = F.softmax(attn_weights, dim=-1)
        attn_output = torch.matmul(attn_probs, v)
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, C)
        return self.out_proj(attn_output)
    # === Конец добавленного на этапе 3 кода ===

# === Добавлено на этапе 4: MLP-блок после attention ===
class FeedForward(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, 4 * embed_dim)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(4 * embed_dim, embed_dim)

    def forward(self, x):
        return self.fc2(self.act(self.fc1(x)))
# === Конец добавленного на этапе 4 кода ===

# === Добавлено как модификация на этапе 8: параллельное ветвление и сверка ===
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.attn = CausalSelfAttention(embed_dim, num_heads)
        self.mlp = FeedForward(embed_dim)
        self.ln_1 = nn.LayerNorm(embed_dim)
        self.ln_2 = nn.LayerNorm(embed_dim)
        self.threshold = 1.0  # можно регулировать чувствительность сверки

    def forward(self, x):
        # параллельные ветви: attention и mlp
        a = self.attn(self.ln_1(x))
        m = self.mlp(self.ln_2(x))

        # сверка: если сильно различаются, подавим результат
        diff = torch.norm(a - m, dim=-1, keepdim=True)  # [B, T, 1]
        mask = (diff < self.threshold).float()
        combined = mask * (a + m) / 2  # если различаются сильно, обнулим

        return x + combined
# === Конец модификации на этапе 8 ===

class MiniGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, max_seq_len=128, num_layers=4):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)

        # === Добавлено на этапе 2: позиционные эмбеддинги ===
        self.position_embedding = nn.Embedding(max_seq_len, embed_dim)
        self.max_seq_len = max_seq_len
        # === Конец добавленного на этапе 2 кода ===

        self.blocks = nn.ModuleList([
            TransformerBlock(embed_dim, num_heads) for _ in range(num_layers)
        ])

        # === Добавлено на этапе 7: финальный LayerNorm ===
        self.final_ln = nn.LayerNorm(embed_dim)
        # === Конец добавленного на этапе 7 кода ===

        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)

        # === Добавлено на этапе 7: Weight tying ===
        self.lm_head.weight = self.token_embedding.weight
        # === Конец добавленного на этапе 7 кода ===

    def forward(self, x, targets=None):
        B, T = x.size()
        tok_emb = self.token_embedding(x)
        pos = torch.arange(0, T, device=x.device).unsqueeze(0)
        pos_emb = self.position_embedding(pos)
        x = tok_emb + pos_emb

        for block in self.blocks:
            x = block(x)

        # === Добавлено на этапе 7: финальный LayerNorm перед головой ===
        x = self.final_ln(x)
        # === Конец добавленного на этапе 7 кода ===

        logits = self.lm_head(x)

        # === Добавлено на этапе 7: расчет loss при наличии targets ===
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        return logits, loss
        # === Конец добавленного на этапе 7 кода ===

    # === Добавлено на этапе 7: функция генерации текста ===
    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.max_seq_len:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx
    # === Конец функции генерации ===

# Пример использования
model = MiniGPT(vocab_size=1000, embed_dim=64, num_heads=4, num_layers=4)
tokens = torch.randint(0, 1000, (2, 5))
targets = torch.randint(0, 1000, (2, 5))
logits, loss = model(tokens, targets)
print("Logits shape:", logits.shape)
print("Loss:", loss.item())

# Пример генерации
generated = model.generate(tokens[:, :2], max_new_tokens=5)
print("Generated tokens:", generated)


Logits shape: torch.Size([2, 5, 1000])
Loss: 45.9717903137207
Generated tokens: tensor([[266, 154, 154, 154, 154, 154, 154],
        [682, 163, 163, 163, 163, 163, 163]])


In [ ]:
# а это у нас уже с 8 и с 9 этапом
import torch
import torch.nn as nn
import torch.nn.functional as F

class CausalSelfAttention(nn.Module):
    # === Добавлено на этапе 3: собственная реализация каузального внимания ===
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5
        self.qkv_proj = nn.Linear(embed_dim, 3 * embed_dim)
        self.out_proj = nn.Linear(embed_dim, embed_dim)

    def forward(self, x):
        B, T, C = x.size()
        qkv = self.qkv_proj(x)
        q, k, v = qkv.chunk(3, dim=-1)
        q = q.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(B, T, self.num_heads, self.head_dim).transpose(1, 2)
        attn_weights = torch.matmul(q, k.transpose(-2, -1)) * self.scale
        mask = torch.tril(torch.ones(T, T, device=x.device)).unsqueeze(0).unsqueeze(0)
        attn_weights = attn_weights.masked_fill(mask == 0, float('-inf'))
        attn_probs = F.softmax(attn_weights, dim=-1)
        attn_output = torch.matmul(attn_probs, v)
        attn_output = attn_output.transpose(1, 2).contiguous().view(B, T, C)
        return self.out_proj(attn_output)
    # === Конец добавленного на этапе 3 кода ===

# === Добавлено на этапе 4: MLP-блок после attention ===
class FeedForward(nn.Module):
    def __init__(self, embed_dim):
        super().__init__()
        self.fc1 = nn.Linear(embed_dim, 4 * embed_dim)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(4 * embed_dim, embed_dim)

    def forward(self, x):
        return self.fc2(self.act(self.fc1(x)))
# === Конец добавленного на этапе 4 кода ===

# === Модифицировано на этапе 9: параллельные блоки attention+MLP ===
class TransformerParallelBlock(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.attn = CausalSelfAttention(embed_dim, num_heads)
        self.mlp = FeedForward(embed_dim)
        self.ln_attn = nn.LayerNorm(embed_dim)
        self.ln_mlp = nn.LayerNorm(embed_dim)

    def forward(self, x):
        a = self.attn(self.ln_attn(x))
        m = self.mlp(self.ln_mlp(x))
        return x + 0.5 * (a + m)
# === Конец изменения на этапе 9 ===

class MiniGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, max_seq_len=128, num_layers=4):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, embed_dim)

        # === Добавлено на этапе 2: позиционные эмбеддинги ===
        self.position_embedding = nn.Embedding(max_seq_len, embed_dim)
        self.max_seq_len = max_seq_len
        # === Конец добавленного на этапе 2 кода ===

        # === Модифицировано на этапе 9: стек параллельных блоков ===
        self.blocks = nn.ModuleList([
            TransformerParallelBlock(embed_dim, num_heads) for _ in range(num_layers)
        ])
        # === Конец модификации на этапе 9 ===

        # === Добавлено на этапе 7: финальный LayerNorm ===
        self.final_ln = nn.LayerNorm(embed_dim)
        # === Конец добавленного на этапе 7 кода ===

        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)

        # === Добавлено на этапе 7: Weight tying ===
        self.lm_head.weight = self.token_embedding.weight
        # === Конец добавленного на этапе 7 кода ===

    def forward(self, x, targets=None):
        B, T = x.size()
        tok_emb = self.token_embedding(x)
        pos = torch.arange(0, T, device=x.device).unsqueeze(0)
        pos_emb = self.position_embedding(pos)
        x = tok_emb + pos_emb

        for block in self.blocks:
            x = block(x)

        # === Добавлено на этапе 7: финальный LayerNorm перед головой ===
        x = self.final_ln(x)
        # === Конец добавленного на этапе 7 кода ===

        logits = self.lm_head(x)

        # === Добавлено на этапе 7: расчет loss при наличии targets ===
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        return logits, loss
        # === Конец добавленного на этапе 7 кода ===

    # === Добавлено на этапе 7: функция генерации текста ===
    @torch.no_grad()
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.max_seq_len:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_token), dim=1)
        return idx
    # === Конец функции генерации ===

# Пример использования
model = MiniGPT(vocab_size=1000, embed_dim=64, num_heads=4, num_layers=4)
tokens = torch.randint(0, 1000, (2, 5))
targets = torch.randint(0, 1000, (2, 5))
logits, loss = model(tokens, targets)
print("Logits shape:", logits.shape)
print("Loss:", loss.item())

# Пример генерации
generated = model.generate(tokens[:, :2], max_new_tokens=5)
print("Generated tokens:", generated)
